### This notebook will examine data from Seattle AirBnB listings and explore the following questions:
1. When is the most affordable time to visit Seattle?
2. Which neighborhoods are the most highly rated, and what is the average price?
3. What kinds of rentals are available in those neighborhoods?

In [1]:
# import libraries and load data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

cal_df = pd.read_csv('./data/calendar.csv')
list_df = pd.read_csv('./data/listings.csv')

In [2]:
cal_df.head()

,listing_id,date,available,price
0,241032,2016-01-04,t,$85.00
1,241032,2016-01-05,t,$85.00
2,241032,2016-01-06,f,NaN
3,241032,2016-01-07,f,NaN
4,241032,2016-01-08,f,NaN


In [3]:
# drop columns with NaN values
price_df = cal_df.dropna()
price_df.dtypes

listing_id     int64
date          object
available     object
price         object
dtype: object

In [4]:
# convert date column from str to datetime
price_df["date"] = pd.to_datetime(price_df["date"], format="%Y-%m-%d")
# convert price column from str to float
price_df["price"] = price_df["price"].replace('[\$,]', '', regex=True).astype(float)
price_df.dtypes

<ipython-input-4-6dc3b4cb6e27>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price_df["date"] = pd.to_datetime(price_df["date"], format="%Y-%m-%d")
<ipython-input-4-6dc3b4cb6e27>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price_df["price"] = price_df["price"].replace('[\$,]', '', regex=True).astype(float)


listing_id             int64
date          datetime64[ns]
available             object
price                float64
dtype: object

In [5]:
# set date as index and groupby month
price_df.index = price_df["date"]
price_df["price"].groupby(by=pd.Grouper(freq='M')).mean().sort_values()

date
2016-01-31    121.568409
2016-02-29    124.293927
2016-03-31    128.644488
2016-04-30    135.097005
2016-11-30    135.688738
2017-01-31    136.882590
2016-10-31    137.031939
2016-12-31    137.251835
2016-05-31    139.538183
2016-09-30    143.255949
2016-06-30    147.473137
2016-08-31    150.656594
2016-07-31    152.094150
Name: price, dtype: float64

### Insight 1
We can now answer our first question. The most affordable time to visit Seattle seems to be in the Winter months - from January through March; the least affordable time to visit Seattle seems to be in the Summer months - from June through August.

In [6]:
list_df.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary',
       'space', 'description', 'experiences_offered', 'neighborhood_overview',
       'notes', 'transit', 'thumbnail_url', 'medium_url', 'picture_url',
       'xl_picture_url', 'host_id', 'host_url', 'host_name', 'host_since',
       'host_location', 'host_about', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood',
       'host_listings_count', 'host_total_listings_count',
       'host_verifications', 'host_has_profile_pic', 'host_identity_verified',
       'street', 'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market',
       'smart_location', 'country_code', 'country', 'latitude', 'longitude',
       'is_location_exact', 'property_type', 'room_type', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', '

In [7]:
# select columns we need for our analysis
hood_df = list_df.loc[:,("price", "property_type", "room_type","neighbourhood_group_cleansed", "review_scores_location")].dropna()
hood_df.head()

,price,property_type,room_type,neighbourhood_group_cleansed,review_scores_location
0,$85.00,Apartment,Entire home/apt,Queen Anne,9.0
1,$150.00,Apartment,Entire home/apt,Queen Anne,10.0
2,$975.00,House,Entire home/apt,Queen Anne,10.0
4,$450.00,House,Entire home/apt,Queen Anne,9.0
5,$120.00,House,Private room,Queen Anne,10.0


In [8]:
# convert price column from str to float
hood_df["price"] = hood_df["price"].replace('[\$,]', '', regex=True).astype(float)
hood_df.dtypes

price                           float64
property_type                    object
room_type                        object
neighbourhood_group_cleansed     object
review_scores_location          float64
dtype: object

In [9]:
hood_df.loc[:,("neighbourhood_group_cleansed", "price", "review_scores_location")].groupby \
(by=["neighbourhood_group_cleansed"]).mean().sort_values(by=["review_scores_location"], ascending=False).head()

,price,review_scores_location
neighbourhood_group_cleansed,,
Capitol Hill,129.372549,9.823529
Downtown,154.280702,9.769737
Ballard,123.647668,9.766839
West Seattle,128.993827,9.765432
Queen Anne,161.591837,9.751020


### Insight 2
We can now answer our second question. The most highly rated neighborhoods are Capitol Hill, Downtown, Ballard, West Seattle, and Queen Anne. The average price of a listing in those neighborhoods ranges between $124-162 per night, with Ballard being the most affordable, followed by West Seattle and Capitol Hill.

In [10]:
# select listings from the highest rated neighborhoods from the previous step
best_neighborhoods = ["Capitol Hill", "Downtown", "Ballard", "West Seattle", "Queen Anne"]
best_df = hood_df[hood_df["neighbourhood_group_cleansed"].isin(best_neighborhoods)]
best_df["neighbourhood_group_cleansed"].value_counts()

Capitol Hill    459
Downtown        456
Queen Anne      245
Ballard         193
West Seattle    162
Name: neighbourhood_group_cleansed, dtype: int64

In [11]:
best_df.property_type.groupby(by=best_df.neighbourhood_group_cleansed).value_counts(normalize=True)

neighbourhood_group_cleansed  property_type  
Ballard                       House              0.647668
                              Apartment          0.269430
                              Townhouse          0.031088
                              Bed & Breakfast    0.010363
                              Camper/RV          0.010363
                              Condominium        0.010363
                              Loft               0.010363
                              Bungalow           0.005181
                              Cabin              0.005181
Capitol Hill                  Apartment          0.644880
                              House              0.289760
                              Condominium        0.023965
                              Townhouse          0.015251
                              Bed & Breakfast    0.010893
                              Cabin              0.004357
                              Loft               0.004357
                          

### Insight 3
We can now answer our third question.  
Ballard listings are 65% Houses, 27% Apartments, with the remainder being other property types.  
Capitol Hill listings are 65% Apartments, 29% Houses, with the remainder being other property types.  
Downtown listings are 88% Apartments, 7% Condos, with the remainder being other property types.  
Queene Anne listings are 58% Apartments, 34% Houses, with the remainder being other property types.  
West Seattle listings are 71% Houses, 20% Apartments, with the remainder being other property types.  